In [1]:
from __future__ import division
from api import git_access,api_access
from git_understand import git_understand as git_understand
from git_log import git2repo
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import networkx as nx
import re
from git_log import git2data,git_commit_info,release_mine
import threading
from threading import Barrier
from multiprocessing import Queue
from os.path import dirname as up
import os
import platform
from commit_guru.cas_manager import *

In [ ]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    data_path = os.getcwd() + '/Test_projects.csv'
else:
    data_path = os.getcwd() + '\\Test_projects.csv'
code_path = os.getcwd()
project_list = pd.read_csv(data_path)
for i in range(project_list.shape[0]):
    try:
        print("I am here")
        understand_source = []
        last_analyzed = None
        access_token = project_list.loc[i,'access_token']
        repo_owner = project_list.loc[i,'repo_owner']
        source_type = project_list.loc[i,'source_type']
        git_url = project_list.loc[i,'git_url']
        api_base_url = project_list.loc[i,'api_base_url']
        repo_name = project_list.loc[i,'repo_name']
        repo_lang = project_list.loc[i,'lang']
        understand_source.append([1,repo_name,git_url,last_analyzed])
        understand_source_df = pd.DataFrame(understand_source,columns = ['id','name','url','last_analyzed'])
        cas_manager = CAS_Manager(understand_source_df)
        cas_manager.start()
        git_data = git2data.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
        #git_data = release_mine.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
        git_data.create_data()
        os.chdir(code_path)
        df_commit = pd.read_pickle(up(code_path) + '/data/commit/' + repo_name + '_commit.pkl')
        df_commit_guru = pd.read_csv(up(code_path) + '/data/commit_guru/' + repo_name + '.csv')
        df_commit_guru_subset = df_commit_guru[['commit_hash','contains_bug']].fillna(False)
        df = pd.merge(df_commit,df_commit_guru_subset, on='commit_hash',how='inner')
        df.to_pickle(up(code_path) + '/data/commit/' + repo_name + '_commit.pkl')
        get_matrix = git_understand.MetricsGetter(git_url,repo_name,repo_lang)
        matrix = get_matrix.get_defective_pair_metrics()
    except ValueError as e:
        print("Exception occured for ",project_list.loc[i,'git_url'])
        print(e)
        continue

In [ ]:
cas_manager = CAS_Manager()
cas_manager.start()

In [ ]:
for i in range(project_list.shape[0]):
    repo_name = project_list.loc[i,'repo_name']
    df_commit = pd.read_pickle('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit/' + repo_name + '_commit.pkl')
    df_commit_guru = pd.read_csv('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/' + repo_name + '.csv')
    df_commit_guru_subset = df_commit_guru[['commit_hash','contains_bug']].fillna(False)
    df = pd.merge(df_commit,df_commit_guru_subset, on='commit_hash',how='inner')
    df.to_pickle('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit/' + repo_name + '_commit.pkl')


In [2]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    data_path = os.getcwd() + '/Test_projects.csv'
else:
    data_path = os.getcwd() + '\\Test_projects.csv'

project_list = pd.read_csv(data_path)
print(project_list)

for i in range(project_list.shape[0]):
    try:
        repo_url = project_list.loc[i,'git_url']
        repo_name = project_list.loc[i,'repo_name']
        repo_lang = project_list.loc[i,'lang']
        get_matrix = git_understand.MetricsGetter(repo_url,repo_name,repo_lang)
        #commits = get_matrix.read_commits()
        #matrix = get_matrix.run()
        matrix = get_matrix.get_defective_pair_metrics()
    except ValueError:
        print("Error")

     repo_name   repo_owner                                         git_url  \
0  thingsboard  thingsboard  https://github.com/thingsboard/thingsboard.git   

            api_base_url  source_type  \
0  http://api.github.com  github_repo   

                               access_token  lang  
0  88dec2f37347858212890803211cef7bed610562  java  
0 ('19d3afa650f6796cfd261bab03c67331e0a601ca', 'f4b32ba333e1938306a729edc1c59857f639dd54')
1 ('19d3afa650f6796cfd261bab03c67331e0a601ca', '84bbd412db0cdfb6975660ba06befb5f0d38bd31')
2 ('87c665f470617f3e06bd08dc98330fdd02849b86', 'c7f55666a8c0e608e4f1fe37e003c0f0265c5779')
6 ('58ec7a335af732810cad08cca96e109a44a7d6a2', 'f6595abbed13f0337cfb0b1743d51f8a7931586e')
7 ('163847cddbf788db825e50b4322ffcf7851bebd6', 'd8443495e890d7475386c3547658c6b91951dc96')
8 ('54215215c5b60c0bb17b5114b139016d74430b40', 'dc188605d4d988085f2a811b6b275e00a30e854b')
9 ('dc188605d4d988085f2a811b6b275e00a30e854b', '6ff2d649741b0579d6fb591e1cfa3d1706649b84')


In [ ]:
df_new = pd.merge(matrix,df_commit_guru, on='commit_hash',how='inner')
df_new = df_new.dropna()
df_new.to_csv('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru_understand/uhabits.csv',index=False)

In [ ]:
matrix_1 = matrix.drop(['Name','commit_hash'],axis = 1)
matrix_1 = matrix_1.drop_duplicates()
matrix_1.shape

In [ ]:
matrix_1.columns

In [ ]:
os.getcwd()